In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gdz-elektrik-datathon-2024/holidays.csv
/kaggle/input/gdz-elektrik-datathon-2024/sample_submission.csv
/kaggle/input/gdz-elektrik-datathon-2024/weather.csv
/kaggle/input/gdz-elektrik-datathon-2024/train.csv
/kaggle/input/gdz-elektrik-datathon-2024/test.csv


In [2]:
# from mpl_toolkits.mplot3d import Axes3D
# from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
# pd.options.mode.chained_assignment = None  # default='warn'
# import warnings
# warnings.filterwarnings('ignore')
from datetime import datetime

In [3]:
train_df = pd.read_csv(r"/kaggle/input/gdz-elektrik-datathon-2024/train.csv")
test_df = pd.read_csv(r"/kaggle/input/gdz-elektrik-datathon-2024/test.csv")
holidays_df = pd.read_csv(r"/kaggle/input/gdz-elektrik-datathon-2024/holidays.csv")
weather_df = pd.read_csv(r"/kaggle/input/gdz-elektrik-datathon-2024/weather.csv")
sample_submission = pd.read_csv(r"/kaggle/input/gdz-elektrik-datathon-2024/sample_submission.csv")

In [4]:
print([len(train_df[train_df['ilce'] == ilce]) for ilce in train_df['ilce'].unique()])

[1106, 1105, 1086, 1120, 1124, 1115, 1125, 1071, 1119, 1086, 920, 856, 1100, 1089, 1085, 1118, 914, 1097, 1124, 1125, 1119, 1124, 1111, 1107, 1124, 1122, 1126, 1119, 938, 566, 1059, 950, 805, 1126, 1027, 1105, 1123, 993, 1086, 1121, 1125, 698, 673, 783, 872, 1039, 622]


In [5]:
print([len(test_df[test_df['ilce'] == ilce]) for ilce in test_df['ilce'].unique()])

[29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29]


In [6]:
ilce_df_array = [[]] * 47
for i, ilce in enumerate(train_df['ilce'].unique()):
    ilce_df_array[i] = train_df[train_df['ilce'] == ilce]
    
min_date = train_df['tarih'].min()
print('min_date :', min_date)
max_date = train_df['tarih'].max()
print('max_date :', max_date)
full_date_range = pd.date_range(start=min_date, end=max_date)
full_date_range = [date.date() for date in full_date_range]
full_date_range = [str(date) for date in full_date_range]

zero_data = []
for ilce_df in ilce_df_array:

    existing_dates = set(ilce_df['tarih'])
    
    missing_dates = [date for date in full_date_range if date not in existing_dates]
    
    # missing_dates = [str(date.date()) for date in missing_dates] # set ['2021-01-01', '2021-01-02', '2021-01-03'] format
    
    for missing_date in missing_dates:
        zero_data.append({'tarih': missing_date, 'ilce': ilce_df['ilce'].values[0], 'bildirimli_sum': 0, 'bildirimsiz_sum': 0})

min_date : 2021-01-01
max_date : 2024-01-31


In [7]:
zero_data = pd.DataFrame(zero_data)
train_df = pd.concat([train_df, zero_data], ignore_index=True)

In [8]:
ilce_df_array = [[]] * 47
for i, ilce in enumerate(train_df['ilce'].unique()):
    ilce_df_array[i] = train_df[train_df['ilce'] == ilce]
    
print([len(train_df[train_df['ilce'] == ilce]) for ilce in train_df['ilce'].unique()])

[1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126, 1126]


In [9]:
# fig = plt.figure(figsize=(25, 3))
# sns.lineplot(data=ilce_df_array[0], x='tarih', y='bildirimsiz_sum', marker='')
# plt.xlabel('date')
# plt.ylabel('plansız kesinti')
# plt.show()

# Modelling

In [10]:
train_df = train_df.drop(['bildirimli_sum'], axis=1)

In [11]:
train_df.head()

,tarih,ilce,bildirimsiz_sum
0,2021-01-01,izmir-aliaga,5
1,2021-01-02,izmir-aliaga,13
2,2021-01-03,izmir-aliaga,4
3,2021-01-04,izmir-aliaga,9
4,2021-01-05,izmir-aliaga,2


In [12]:
!pip show autogluon

In [ ]:
# !pip install scikit-learn==1.4.0
!pip install autogluon==1.1.0

In [14]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

train_tsdf = TimeSeriesDataFrame.from_data_frame(
    train_df,
    id_column="ilce",
    timestamp_column="tarih"
)

predictor = TimeSeriesPredictor(
    freq="D",
    prediction_length=29,
    path="autogluon-m4-hourly_1",
    target="bildirimsiz_sum",
    eval_metric="MASE",
)

predictor.fit(
    train_tsdf,
    presets="medium_quality",
    time_limit=600,
)

Beginning AutoGluon training... Time limit = 600s
AutoGluon will save models to 'autogluon-m4-hourly_1'
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Dec 19 13:14:11 UTC 2023
CPU Count:          4
GPU Count:          0
Memory Avail:       30.10 GB / 31.36 GB (96.0%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Setting presets to: medium_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MASE,
 'freq': 'D',
 'hyperparameters': 'light',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 29,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'bildirimsiz_sum',
 'time_limit': 600,
 'verbosity': 2}

Provided train_data has 52922 rows, 47 time series. Median time series 

In [15]:
predictor

In [16]:
predictions = predictor.predict(train_tsdf)

Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [17]:
predictions

mean       0.1       0.2       0.3       0.4  \
item_id        timestamp                                                      
izmir-aliaga   2024-02-01  5.201255  2.521175  3.129802  3.821094  4.529765   
               2024-02-02  5.453587  2.417618  3.234872  3.982102  4.814445   
               2024-02-03  5.253850  2.028794  2.933024  3.768392  4.571925   
               2024-02-04  5.282825  2.002696  2.927203  3.822844  4.559482   
               2024-02-05  6.083039  2.315224  3.259524  4.295290  5.160411   
...                             ...       ...       ...       ...       ...   
manisa-ahmetli 2024-02-25  0.661113 -0.472557 -0.100965  0.176371  0.428798   
               2024-02-26  0.827956 -0.440011 -0.033995  0.247563  0.566979   
               2024-02-27  0.821923 -0.460757 -0.040586  0.237516  0.558913   
               2024-02-28  0.819030 -0.471126 -0.039026  0.235274  0.555920   
               2024-02-29  0.839703 -0.449016 -0.009800  0.262902  0.577993   

                                0.5       0.6       0.7        0.8        0.9  
item_id        timestamp                                                       
izmir-aliaga   2024-02-01  5.201255  5.999046  7.127612   8.613727  11.241096  
               2024-02-02  5.453587  6.208048  7.296566   8.893885  11.322761  
               2024-02-03  5.253850  6.013094  7.041426   8.652251  11.016119  
               2024-02-04  5.282825  6.024094  7.008616   8.646334  10.989678  
               2024-02-05  6.083039  6.924552  8.078605  10.019071  12.698942  
...                             ...       ...       ...        ...        ...  
manisa-ahmetli 2024-02-25  0.661113  0.962890  1.250880   1.603337   2.253626  
               2024-02-26  0.827956  1.166926  1.463008   1.879587   2.662738  
               2024-02-27  0.821923  1.156260  1.453279   1.868588   2.641900  
               2024-02-28  0.819030  1.152004  1.448753   1.860268   2.612892  
               2024-02-29  0.839703  1.170922  1.467223   1.870406   2.595724  

[1363 rows x 10 columns]

In [18]:
sample_submission

,unique_id,bildirimsiz_sum
0,2024-02-01-izmir-aliaga,NaN
1,2024-02-01-izmir-bayindir,NaN
2,2024-02-01-izmir-bayrakli,NaN
3,2024-02-01-izmir-bergama,NaN
4,2024-02-01-izmir-bornova,NaN
...,...,...
1358,2024-02-29-izmir-beydag,NaN
1359,2024-02-29-izmir-narlidere,NaN
1360,2024-02-29-izmir-selcuk,NaN
1361,2024-02-29-manisa-kula,NaN


In [19]:
predictions.loc['izmir-aliaga', '2024-02-01']

mean     5.201255
0.1      2.521175
0.2      3.129802
0.3      3.821094
0.4      4.529765
0.5      5.201255
0.6      5.999046
0.7      7.127612
0.8      8.613727
0.9     11.241096
Name: (izmir-aliaga, 2024-02-01 00:00:00), dtype: float64

In [ ]:
for _id in sample_submission['unique_id']:
    tarih = _id[0:10]
    ilce = _id[11:len(_id)]
    try:
        sample_submission.loc[sample_submission['unique_id'] == _id, 'bildirimsiz_sum'] = predictions.loc[ilce, tarih][4] # 0: mean
    except:
        print(ilce, tarih)
        sample_submission.loc[sample_submission['unique_id'] == _id, 'bildirimsiz_sum'] = 3

In [21]:
sample_submission.loc[sample_submission['bildirimsiz_sum'] < 0, 'bildirimsiz_sum'] = 0

In [22]:
output = pd.DataFrame({
    'unique_id': sample_submission['unique_id'],
    'bildirimsiz_sum': sample_submission['bildirimsiz_sum']
})
output.to_csv('kaggle_base_comprehension_sub.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
